# resize dengan padding

In [1]:
import cv2
import numpy as np
import os

# Path folder input dan output
input_path = 'Keriting/'  # Ganti dengan folder Anda
output_path = 'Keriting_resized/'  # Ganti folder output
os.makedirs(output_path, exist_ok=True)

target_size = (224, 224)  # Ukuran target (sesuai model)

# Fungsi resize dengan padding
def resize_with_padding(image, target_size):
    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    
    # Buat kanvas dengan ukuran target dan tambahkan padding
    canvas = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    pad_top = (target_size[0] - new_h) // 2
    pad_left = (target_size[1] - new_w) // 2
    canvas[pad_top:pad_top+new_h, pad_left:pad_left+new_w] = resized_image
    return canvas

# Resize semua gambar dalam folder
for file_name in os.listdir(input_path):
    file_path = os.path.join(input_path, file_name)
    img = cv2.imread(file_path)
    if img is not None:
        resized_img = resize_with_padding(img, target_size)
        cv2.imwrite(os.path.join(output_path, file_name), resized_img)
    else:
        print(f"File tidak dapat dibaca: {file_path}")


# resize tanpa padding

In [7]:
import os
import cv2

# Path folder input (Dataset asli)
input_path = 'Layu/'  
# Path folder output (Dataset setelah resize)
output_path = 'Layu_resized/'  

# Pastikan folder output dibuat jika belum ada
os.makedirs(output_path, exist_ok=True)

# Format gambar yang valid
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

# Fungsi untuk meresize gambar
def resize_images_in_folder(input_dir, output_dir, target_size=(224, 224)):
    for root, _, files in os.walk(input_dir):  # Telusuri folder dan subfolder
        for file in files:
            if file.lower().endswith(valid_extensions):  # Hanya proses file gambar
                file_path = os.path.join(root, file)  # Path file asli
                relative_path = os.path.relpath(file_path, input_dir)  # Path relatif dari folder input
                output_file_path = os.path.join(output_dir, relative_path)  # Path tujuan file
                os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Buat folder tujuan jika perlu

                # Baca dan resize gambar
                img = cv2.imread(file_path)
                if img is not None:
                    resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_CUBIC)
                    cv2.imwrite(output_file_path, resized_img)  # Simpan gambar hasil resize
                else:
                    print(f"Gagal membaca gambar: {file_path}")

# Jalankan fungsi untuk meresize semua gambar
resize_images_in_folder(input_path, output_path)


[INFO] File diproses: After_Split/train\Keriting\IMG20241127171041_01.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171042.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171043.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171043_01.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171053.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171054.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171055.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171056.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171056_01.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171057.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Keriting\IMG20241127171058.jpg | Augmented: 5
[INFO] File diproses: After_Split/train\Kerit

# Augmentasi

In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

input_path = 'After_Split/train'  # Folder input
output_path = 'Augmented_dataset_After_Split/'  # Folder output
os.makedirs(output_path, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=0.1
)

def augment_and_save_images(input_folder, output_folder, num_augmented_images=5):
    total_processed_files = 0
    total_augmented_images = 0
    saved_files = []  # List untuk menyimpan nama file yang dihasilkan
    
    for category in os.listdir(input_folder):
        category_path = os.path.join(input_folder, category)
        augmented_category_path = os.path.join(output_folder, category)
        os.makedirs(augmented_category_path, exist_ok=True)

        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            try:
                img = load_img(file_path)
                img_array = img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                i = 0
                for batch in datagen.flow(
                    img_array,
                    batch_size=1,
                    save_to_dir=augmented_category_path,
                    save_prefix='aug',
                    save_format='jpg'
                ):
                    i += 1
                    total_augmented_images += 1

                    # Simpan nama file yang dihasilkan
                    for generated_file in os.listdir(augmented_category_path):
                        if generated_file not in saved_files:
                            saved_files.append(generated_file)

                    if i >= num_augmented_images:
                        break
                total_processed_files += 1
            except Exception as e:
                print(f"[ERROR] Gagal memproses {file_path}: {e}")

    print("\n===== Summary =====")
    print(f"Total gambar asli diproses: {total_processed_files}")
    print(f"Total gambar augmentasi (dihitung): {total_augmented_images}")
    print(f"Total gambar augmentasi (disimpan): {len(saved_files)}")

augment_and_save_images(input_path, output_path, num_augmented_images=5)



===== Summary =====
Total gambar asli diproses: 423
Total gambar augmentasi (dihitung): 2115
Total gambar augmentasi (disimpan): 1905


In [12]:
import os

input_path = 'After_Split/train'
total_images = sum([len(files) for _, _, files in os.walk(input_path)])
print(f"Total gambar asli: {total_images}")


Total gambar asli: 423


In [13]:
output_path = 'Augmented_dataset_After_Split/'
total_augmented_images = sum([len(files) for _, _, files in os.walk(output_path)])
print(f"Total gambar setelah augmentasi: {total_augmented_images}")


Total gambar setelah augmentasi: 2046


In [25]:
import os

output_path = 'Augmented_dataset_After_Split/'
file_set = set()
duplicate_files = []

for root, _, files in os.walk(output_path):
    for file in files:
        if file in file_set:
            duplicate_files.append(os.path.join(root, file))
        else:
            file_set.add(file)

print(f"Total file unik: {len(file_set)}")
print(f"Total file duplikat: {len(duplicate_files)}")
if duplicate_files:
    print("File duplikat ditemukan:", duplicate_files)


Total file unik: 0
Total file duplikat: 0


In [9]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import uuid

# === Konfigurasi Dataset ===
original_dataset_path = 'Dataset_lengkap_2 - Resized/'  # Folder dataset asli
augmented_dataset_path = 'Dataset_lengkap_2 - Resized/augmented/'  # Folder output augmentasi
target_size = (224, 224)  # Ukuran target gambar
augmentation_per_image = 5  # Jumlah augmentasi per gambar

# === Buat Folder Output Baru ===
os.makedirs(augmented_dataset_path, exist_ok=True)

# === Konfigurasi Augmentasi ===
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# === Proses Augmentasi ===
total_original_images = 0
total_augmented_images = 0

for category in os.listdir(original_dataset_path):
    category_path = os.path.join(original_dataset_path, category)
    if not os.path.isdir(category_path):
        continue  # Skip jika bukan folder
    
    augmented_category_path = os.path.join(augmented_dataset_path, category)
    os.makedirs(augmented_category_path, exist_ok=True)  # Buat folder kategori output
    
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        if not img_file.lower().endswith(('jpg', 'jpeg', 'png')):
            continue  # Skip jika bukan file gambar
        
        # Load gambar asli
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = img_array.reshape((1,) + img_array.shape)  # Tambahkan dimensi batch
        
        # Augmentasi gambar
        total_original_images += 1
        i = 0
        for batch in datagen.flow(
            img_array,
            batch_size=1,
            save_to_dir=augmented_category_path,
            save_prefix=f'aug_{uuid.uuid4()}',
            save_format='jpg'
        ):
            i += 1
            total_augmented_images += 1
            if i >= augmentation_per_image:
                break  # Hentikan setelah jumlah augmentasi per gambar tercapai

print(f"Proses augmentasi selesai.")
print(f"Total gambar asli diproses: {total_original_images}")
print(f"Total gambar augmentasi yang dihasilkan: {total_augmented_images}")


Proses augmentasi selesai.
Total gambar asli diproses: 529
Total gambar augmentasi yang dihasilkan: 2645


In [1]:
import os
import uuid
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# === Konfigurasi Dataset ===
original_dataset_path = 'all_augmented/after_split/'  # Folder dataset asli
augmented_dataset_path = '"all_augmented/after_split/all_augmented'  # Folder output augmentasi
target_size = (224, 224)  # Ukuran target gambar

# === Buat Folder Output Baru ===
os.makedirs(augmented_dataset_path, exist_ok=True)

# Fungsi untuk menerapkan augmentasi individual
def augment_image(img, augmentation, category_path, category):
    # Terapkan augmentasi
    datagen = ImageDataGenerator(**augmentation)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)  # Tambahkan dimensi batch

    # Simpan hasil augmentasi
    for batch in datagen.flow(
        img_array,
        batch_size=1,
        save_to_dir=category_path,
        save_prefix=f'{category}_aug_{uuid.uuid4()}',
        save_format='jpg',
    ):
        break  # Hanya simpan satu gambar untuk setiap augmentasi

# Konfigurasi augmentasi individual
augmentations = [
    {"rotation_range": 30},
    {"width_shift_range": 0.2},
    {"height_shift_range": 0.2},
    {"zoom_range": 0.2},
    {"brightness_range": [0.8, 1.2]},  # Brightness augmentation menggantikan horizontal flip
    {"fill_mode": "nearest"},
]

# === Proses Augmentasi ===
total_original_images = 0
total_augmented_images = 0

for category in os.listdir(original_dataset_path):
    category_path = os.path.join(original_dataset_path, category)
    if not os.path.isdir(category_path):
        continue  # Skip jika bukan folder
    
    augmented_category_path = os.path.join(augmented_dataset_path, category)
    os.makedirs(augmented_category_path, exist_ok=True)  # Buat folder kategori output
    
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        if not img_file.lower().endswith(('jpg', 'jpeg', 'png')):
            continue  # Skip jika bukan file gambar
        
        # Load gambar asli
        img = load_img(img_path, target_size=target_size)
        
        # Terapkan setiap augmentasi
        for aug in augmentations:
            augment_image(img, aug, augmented_category_path, category)
            total_augmented_images += 1

        total_original_images += 1

print(f"Proses augmentasi selesai.")
print(f"Total gambar asli diproses: {total_original_images}")
print(f"Total gambar augmentasi yang dihasilkan: {total_augmented_images}")


Proses augmentasi selesai.
Total gambar asli diproses: 500
Total gambar augmentasi yang dihasilkan: 3000


In [3]:
import os
import numpy as np
import uuid
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img

# === Konfigurasi Path ===
dataset_path = "all_augmented/after_split"  # Path dataset utama (train, test, val)
target_size = (224, 224)  # Resolusi target gambar

# Fungsi untuk menyimpan hasil augmentasi di folder yang sama
def save_augmented_image(img_array, save_path, prefix):
    img_array = np.clip(img_array, 0, 255).astype('uint8')  # Pastikan nilai dalam range valid
    img = array_to_img(img_array)
    img.save(os.path.join(save_path, f"{prefix}_{uuid.uuid4()}.jpg"))

# === Proses Augmentasi ===
total_original_images = 0
total_augmented_images = 0

# Iterasi setiap subset (train, test, val)
for subset in ["train", "test", "validation"]:
    subset_path = os.path.join(dataset_path, subset)
    for category in os.listdir(subset_path):
        category_path = os.path.join(subset_path, category)
        if not os.path.isdir(category_path):
            continue  # Skip jika bukan folder

        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue  # Skip jika bukan file gambar

            # Load gambar asli
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)

            # === Augmentasi ===
            # Flip Horizontal
            flipped_img = np.fliplr(img_array)
            save_augmented_image(flipped_img, category_path, "flip")

            # 90° Rotate Clockwise
            rotated_90_cw = np.rot90(img_array, k=-1)
            save_augmented_image(rotated_90_cw, category_path, "rot90_cw")

            # 90° Rotate Counter-Clockwise
            rotated_90_ccw = np.rot90(img_array, k=1)
            save_augmented_image(rotated_90_ccw, category_path, "rot90_ccw")

            # Random Rotation (-15° to +15°)
            for angle in [-15, 15]:
                rotated_img = np.clip(np.rot90(img_array, k=angle), 0, 255)
                save_augmented_image(rotated_img, category_path, f"rot_{angle}")

            # Brightness Adjustment (-15% to +15%)
            brightness_adjustments = [0.85, 1.15]
            for factor in brightness_adjustments:
                bright_img = img_array * factor
                save_augmented_image(bright_img, category_path, f"bright_{int(factor * 100)}")

            # Exposure Adjustment (-10% to +10%)
            exposure_adjustments = [0.9, 1.1]
            for factor in exposure_adjustments:
                exposed_img = img_array * factor
                save_augmented_image(exposed_img, category_path, f"expose_{int(factor * 100)}")

            total_original_images += 1
            total_augmented_images += 6  # 6 augmentasi per gambar asli

print(f"Proses augmentasi selesai.")
print(f"Total gambar asli diproses: {total_original_images}")
print(f"Total gambar augmentasi yang dihasilkan: {total_augmented_images}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'all_augmented/after_split\\val'

In [1]:
import os

def count_images_in_folder(folder_path):
    # Menghitung jumlah gambar dalam setiap kelas di folder tertentu
    class_counts = {}
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            # Menghitung jumlah gambar dalam setiap kelas
            image_count = len([f for f in os.listdir(class_folder) if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif'))])
            class_counts[class_name] = image_count
    return class_counts

def count_images_in_dataset(base_path):
    # Menghitung jumlah gambar untuk train, test, dan val
    dataset_summary = {}
    
    for subset in ['train', 'test', 'valid']:
        subset_path = os.path.join(base_path, subset)
        print(f"\n{subset.capitalize()} Set:")
        class_counts = count_images_in_folder(subset_path)
        for class_name, count in class_counts.items():
            print(f"Class {class_name}: {count} images")
        dataset_summary[subset] = class_counts
    
    return dataset_summary

# Ganti dengan path ke dataset Anda
base_path = "cabe_final/"  # Ganti dengan path folder dataset Anda
dataset_summary = count_images_in_dataset(base_path)



Train Set:
Class Keriting: 402 images
Class Kuning: 414 images
Class Layu: 384 images

Test Set:
Class Keriting: 17 images
Class Kuning: 17 images
Class Layu: 16 images

Valid Set:
Class Keriting: 16 images
Class Kuning: 18 images
Class Layu: 16 images


In [2]:
import os

def count_images_in_folder(folder_path):
    # Menghitung jumlah gambar dalam setiap kelas di folder tertentu
    class_counts = {}
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):
            # Menghitung jumlah gambar dalam setiap kelas
            image_count = len([f for f in os.listdir(class_folder) if f.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif'))])
            class_counts[class_name] = image_count
    return class_counts

def count_images_in_dataset(base_path):
    # Menghitung jumlah gambar untuk train, test, dan val
    dataset_summary = {}
    
    for subset in ['train', 'test', 'validation']:
        subset_path = os.path.join(base_path, subset)
        print(f"\n{subset.capitalize()} Set:")
        class_counts = count_images_in_folder(subset_path)
        for class_name, count in class_counts.items():
            print(f"Class {class_name}: {count} images")
        dataset_summary[subset] = class_counts
    
    return dataset_summary

# Ganti dengan path ke dataset Anda
base_path = "all_augmented_final/"  # Ganti dengan path folder dataset Anda
dataset_summary = count_images_in_dataset(base_path)



Train Set:
Class Keriting: 903 images
Class Kuning: 987 images
Class Layu: 910 images

Test Set:
Class Keriting: 119 images
Class Kuning: 126 images
Class Layu: 105 images

Validation Set:
Class Keriting: 147 images
Class Kuning: 98 images
Class Layu: 105 images
